# Hopfield Networks and Boltzmann Machines:

## Hopfield Networks:

### Content Addressable Memory or Auto-Associative Memory:

Inspired by the ability of humans to retrieve something from memory, given a small part of the data as a prompt.

Eg. "to be or not to be, ..."

### Energy Based Models:

<table>
    <tr>
        <td>            
            <img src="images/energy-function-noisy-images.png" width="100%">
        </td>
        <td>
            <img src="images/energy-function-for-images.png" width="100%">
        </td>
    <tr>
</table>


Define an *energy function* on the set of images that maps states to an energy value, such that for those images the energy value is very low.

If we start with a corrupted or noisy image and make small changes to the image, we descend the *basin of attraction*.

Consider the set of all images of $d$ pixels each, where every pixel is either white or black, so we say $x_j=-1$ if pixel $j$ is black and $x_j=1$ if pixel $j$ is white, for all $1\leq j\leq d$.


$$
    E(x) = - \big( \underbrace{\frac{1}{2} \sum_{i, j} x_i w_{ij} x_j}_{\text{Consider pixel pairs } x_i, x_j} + \underbrace{\sum_{i} b_i x_i}_{\text{Effect of pixel }x_i \text{ on energy}} \big) \tag{1}
$$

If $x_i$ and $x_j$ are the same colour, we subtract $w_{ij}$. If they are opposite, we add $w_{ij}$.

If $w_{ij}$ is positive, that means images for which the pixel colours $x_i$ and $x_j$ are the same will have a lower energy value &mdash; since $w_{ij}$ would be subtracted in $(1)$, therefore resulting in a lower energy value.

If $w_{ij}$ is negative, then images for which the pixel colours $x_i$ and $x_j$ are different will result in $w_{ij}$ being subtracted from the energy value.

$w_{ij}$ 'measures' the correlation between pixels $i$ and $j$.

Just before 13:00

### Constraint Satisfaction Problems:

Suppose you have to place $n$ queens on an $n\times n$ board such that no two queens are attacking each other.

<table>
    <tr>
        <td>            
            <img src="images/chess-queen-constraint-satisfaction-problem.png" width="100%">
        </td>
        <td>
            <p style="text-align: center;">Iterative improvement / Local search</p>
            <img src="images/chess-queen-iterative-improvement.png" width="100%">
        </td>
    <tr>
</table>


Solving the constraint satisfcation problem by local search involves randomly initialising the variables (eg. randomly distributing the queens), then varying one variable at a time, trying to reduce the number of violations at each step.

### Resources:
- <a href="https://medium.com/@serbanliviu/hopfield-nets-and-the-brain-e5880070cdba">Hopfield Networks and the Brain</a>
